In [9]:
import numpy as np

In [10]:
# Function to calculate the block size based on samples per mm and scanning velocity
def calculate_block_size(samples_per_mm, scanning_velocity_mm_s, sample_rate_hz):
    # Calculate how many samples are needed per second based on the scanning velocity and samples per millimeter
    samples_per_s = samples_per_mm * scanning_velocity_mm_s

    # Calculate block size based on the sample rate and the required samples per second
    block_size = sample_rate_hz / samples_per_s
    return block_size

# Constants
MCU_CLOCK_SPEED_HZ = 100e6  # Assume 100 MHz for Cortex-M4
OPERATIONS_PER_SAMPLE = 4    # Assume 2 adds, 1 multiply, 1 window function for the Goertzel algorithm
CHANNELS = 5                # Five channels running in parallel
SAFETY_FACTOR = 0.1         # 10% safety margin

# Input parameters
samples_per_mm = 15 / 2  # 15 samples per 2 mm
scanning_velocity_mm_s = 40  # Scanning velocity in mm/s
sample_rate_hz = 32e3  # Sample rate of the ADC in Hz (max 100 kHz)

### Block Size Calculation
The block size is calculated based on the number of samples needed per millimeter and the scanning velocity. It determines how many samples are processed in one go by the Goertzel algorithm.

$$
\text{Block Size} = \frac{\text{Sample Rate}}{\text{Samples per Second}}
$$

where:
- $\text{Sample Rate}$ is the rate at which the ADC provides samples.
- $\text{Samples per Second}$ is the product of the scanning velocity and the number of samples per millimeter.


### Operations Per Block
The total number of operations required to process one block of data, including all arithmetic operations needed by the Goertzel algorithm.

$$
\text{Operations per Block} = \text{Block Size} \times \text{Operations per Sample}
$$


### Execution Time Per Block
The time required for the MCU to execute all operations for one block of data.

$$
\text{Execution Time per Block} = \frac{\text{Operations per Block}}{\text{MCU Clock Speed}}
$$


### Sample Period
The time interval between consecutive samples, which is the inverse of the sample rate.

$$
\text{Sample Period} = \frac{1}{\text{Sample Rate}}
$$


### Total Execution Time
The cumulative time required for the MCU to process the blocks from all channels.

$$
\text{Total Execution Time} = \text{Execution Time per Block} \times \text{Number of Channels}
$$


### Safety Margin
An additional time added to the total execution time to account for unexpected delays and ensure reliability.

$$
\text{Total Execution Time with Safety Margin} = \text{Total Execution Time} \times (1 + \text{Safety Factor})
$$


### MCU Utilization
The percentage of the MCU's capacity that is used to process the data. It is a ratio of the total execution time (with safety margin) to the sample period.

$$
\text{MCU Utilization} = \left( \frac{\text{Total Execution Time with Safety Margin}}{\text{Sample Period}} \right) \times 100\%
$$


In [11]:
# Calculate block size
block_size = calculate_block_size(samples_per_mm, scanning_velocity_mm_s, sample_rate_hz)

# Functions for calculation
def calculate_operations_per_block(block_size, ops_per_sample):
    return block_size * ops_per_sample

def calculate_execution_time_per_block(operations, clock_speed_hz):
    return operations / clock_speed_hz

def calculate_sample_period(sampling_rate_hz):
    return 1 / sampling_rate_hz

def calculate_total_execution_time(exec_time_per_block, channels):
    return exec_time_per_block * channels

def calculate_safety_margin(total_exec_time, safety_factor):
    return total_exec_time * (1 + safety_factor)

# Calculations
operations_per_block = calculate_operations_per_block(block_size, OPERATIONS_PER_SAMPLE)
execution_time_per_block = calculate_execution_time_per_block(operations_per_block, MCU_CLOCK_SPEED_HZ)
sample_period = calculate_sample_period(sample_rate_hz)
total_execution_time = calculate_total_execution_time(execution_time_per_block, CHANNELS)
total_execution_time_with_margin = calculate_safety_margin(total_execution_time, SAFETY_FACTOR)
# Calculate MCU utilization
mcu_utilization = (total_execution_time_with_margin / sample_period) * 100


In [12]:
# Output the results with MCU utilization
print(f"Calculated block size: {block_size}")
print(f"Operations per block: {operations_per_block}")
print(f"Execution time per block (s): {execution_time_per_block:.10f}")
print(f"Sample period (s): {sample_period:.10f}")
print(f"Total execution time for {CHANNELS} channels (s): {total_execution_time:.10f}")
print(f"Total execution time with safety margin (s): {total_execution_time_with_margin:.10f}")
print(f"MCU utilization: {mcu_utilization:.2f}%")
print(f"Can the MCU handle the load? {'YES' if sample_period > total_execution_time_with_margin else 'NO'}")

Calculated block size: 106.66666666666667
Operations per block: 426.6666666666667
Execution time per block (s): 0.0000042667
Sample period (s): 0.0000312500
Total execution time for 5 channels (s): 0.0000213333
Total execution time with safety margin (s): 0.0000234667
MCU utilization: 75.09%
Can the MCU handle the load? YES
